# Acessando a API do Pipefy e fazendo download de relátorios de forma automática

Importando a biblioteca "resquests" para fazer requisições HTTP em Python.

Importando a biblioteca "json" para fácil conversão de strings em dicionários Python.


In [20]:
import requests
import json

Definindo a URL do site e os headers necessários para o login.

Nessa etapa é necessário substituir a palavra "Token" escrito no código abaixo (logo após "Bearer"), pelo seu token de acesso do Pipefy.

Esse token pode ser coletado no seguinte link "https://app.pipefy.com/tokens".

In [2]:
url = "https://api.pipefy.com/graphql"

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer Token"
}

### Definindo a primeira consulta em GraphQL para extrair o ID de exportação do relatório

Essa primeira consulta nos retorna o ID de exportação do relatório.

Nessa etapa é necessário substituir as palavras "IdPipe" e "IdPipeRelatorio" pelos respectivos IDs do seu pipe e do seu relatório, esse que já deve conter as colunas que deseja, podendo ser editadas na própria página de relatório do Pipefy.

Ambos IDs podem ser identificados no link da página que contem seu relatório, e estarão dispostos da seguinte forma:

https://app.pipefy.com/pipes/Id_do_seu_Pipe/reports_v2/Id_do_seu_relatório

(O link acima não retorna uma página válida, só mostra o padrão de link que será encontrado)

Note que para o ID de seu relatório ser encontrado no link, ele deve ter sido salvo ao menos 1 vez.

In [3]:
payload_mutation = {"query": '''
mutation {

  exportPipeReport(input: { pipeId: IdPipe, pipeReportId: IdPipeRelatorio}) {

    pipeReportExport {

      id

    }

  }

}
'''}

### Fazendo o request para o ID de exportação do relatório

In [6]:
response_mutation = requests.post(url, json=payload_mutation, headers=headers)

Utilizando o atributo text do nosso request é possível ver o conteúdo retornado, porém em formato de texto.

Para esse conteúdo ser acessado de forma fácil precisamos transformalo em um dicionário Python com a biblioteca "json".

Uma vez no formato necessário, podemos acessar o ID de exportação da seguinte forma:

In [46]:
exportID = json.loads(response_mutation.text)['data']['exportPipeReport']['pipeReportExport']['id']

### Definindo a segunda consulta em GraphQL para extrair o link de download do relatório

In [38]:
payload_query = {"query" : '''{

  pipeReportExport(id: idExportacao) {

    fileURL

    state

    startedAt

    requestedBy {

      id

    }

  }

}
'''}

### Substituindo o ID de exportação na consulta acima

In [41]:
payload_query['query'] = payload_query['query'].replace("idExportacao", exportID)

### Fazendo o request para o link de download do relatório

In [43]:
response_query = requests.post(url, json=payload_query, headers=headers)

Fazendo o mesmo processo de conversão do último request, temos o link para download do relatório.

In [45]:
downloadLInk = json.loads(response_query.text)['data']['pipeReportExport']['fileURL']

### Fazendo o request para o link de download

In [35]:
download = requests.get(downloadLInk)

O atributo "content" nos fornece o conteúdo da página em bytes, utilizando a função "open" para manipular arquivos, com o paramêtro "wb", logo após o nome desejado do arquivo, é permitido escrever-los em um novo documento.

Ao utilizar a função "write", passamos como parâmetro o conteúdo em bytes do link de downlaod, escrevendo assim o relatório desejado.

In [36]:
with open("Relatorio.xlsx", "wb") as d:
    d.write(download.content)